# 使用全局向量（GloVe）的词嵌入
:label:`sec_glove`


在上下文窗口内词-词共现
可能携带丰富的语义信息。
例如，
在一个大型语料库中
词"solid"更有可能与"ice"共现
而不是"steam"，
但词"gas"
可能与"steam"的共现频率
比"ice"更高。
此外，
这样的共现的全局语料库统计
可以预先计算：
这可以导致更有效的训练。
为了利用整个语料库中的
统计信息进行词嵌入，
让我们首先回顾
:numref:`subsec_skip-gram`中的skip-gram模型，
但是使用全局语料库统计
如共现计数来解释它。

## 基于全局语料库统计的Skip-Gram
:label:`subsec_skipgram-global`

用$q_{ij}$
表示在skip-gram模型中
给定词$w_i$条件下词$w_j$的条件概率
$P(w_j\mid w_i)$，
我们有

$$q_{ij}=\frac{\exp(\mathbf{u}_j^\top \mathbf{v}_i)}{ \sum_{k \in \mathcal{V}} \exp(\mathbf{u}_k^\top \mathbf{v}_i)},$$

其中
对于任何索引$i$
向量$\mathbf{v}_i$和$\mathbf{u}_i$
分别表示作为中心词和上下文词的词$w_i$，
而$\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$
是词汇表的索引集。

考虑可能在语料库中出现多次的词$w_i$。
在整个语料库中，
所有将$w_i$作为中心词的上下文词
形成一个*多重集*$\mathcal{C}_i$
允许同一元素有多个实例。
对于任意元素，
其实例数量称为其*重数*。
以一个例子说明，
假设词$w_i$在语料库中出现两次
并且在这两个上下文窗口中
以$w_i$为中心词的上下文词的索引
是$k, j, m, k$和$k, l, k, j$。
因此，多重集$\mathcal{C}_i = \{j, j, k, k, k, k, l, m\}$，其中
元素$j, k, l, m$
的重数分别为2, 4, 1, 1。

现在我们用$x_{ij}$表示
多重集$\mathcal{C}_i$中元素$j$的重数。
这是在整个语料库中
词$w_j$（作为上下文词）
和词$w_i$（作为中心词）
在同一上下文窗口中共现的全局共现计数。
使用这样的全局语料库统计，
skip-gram模型的损失函数等价于

$$-\sum_{i\in\mathcal{V}}\sum_{j\in\mathcal{V}} x_{ij} \log\,q_{ij}.$$
:eqlabel:`eq_skipgram-x_ij`

我们进一步用
$x_i$
表示$w_i$作为中心词出现的所有上下文词的数量，
这等同于$|\mathcal{C}_i|$。
令$p_{ij}$
为生成上下文词$w_j$给定中心词$w_i$的条件概率
$x_{ij}/x_i$，
:eqref:`eq_skipgram-x_ij`
可以重写为

$$-\sum_{i\in\mathcal{V}} x_i \sum_{j\in\mathcal{V}} p_{ij} \log\,q_{ij}.$$
:eqlabel:`eq_skipgram-p_ij`

在:eqref:`eq_skipgram-p_ij`中，$-\sum_{j\in\mathcal{V}} p_{ij} \log\,q_{ij}$计算了
全局语料库统计的条件分布$p_{ij}$
和
模型预测的条件分布$q_{ij}$
之间的交叉熵。
该损失
也按上述方式由$x_i$加权。
最小化:eqref:`eq_skipgram-p_ij`中的损失函数
将使预测的条件分布
接近
来自全局语料库统计的条件分布。


尽管通常用于测量
概率分布之间的距离，
交叉熵损失函数在这里可能不是一个好的选择。
一方面，正如我们在:numref:`sec_approx_train`中提到的，
正确归一化$q_{ij}$
的成本会导致对整个词汇表求和，
这可能会非常昂贵。
另一方面，
从大量语料库中提取的大量稀有事件
通常被建模为通过交叉熵损失
分配过多权重。

## GloVe模型

鉴于此，
*GloVe*模型基于平方损失对skip-gram模型进行了三项更改 :cite:`Pennington.Socher.Manning.2014`：

1. 使用变量$p'_{ij}=x_{ij}$和$q'_{ij}=\exp(\mathbf{u}_j^\top \mathbf{v}_i)$
不是概率分布
并对两者取对数，所以平方损失项为$\left(\log\,p'_{ij} - \log\,q'_{ij}\right)^2 = \left(\mathbf{u}_j^\top \mathbf{v}_i - \log\,x_{ij}\right)^2$。
2. 为每个词$w_i$添加两个标量模型参数：中心词偏置$b_i$和上下文词偏置$c_i$。
3. 用权重函数$h(x_{ij})$替换每个损失项的权重，其中$h(x)$在区间$[0, 1]$上递增。

综合起来，训练GloVe是为了最小化以下损失函数：

$$\sum_{i\in\mathcal{V}} \sum_{j\in\mathcal{V}} h(x_{ij}) \left(\mathbf{u}_j^\top \mathbf{v}_i + b_i + c_j - \log\,x_{ij}\right)^2.$$
:eqlabel:`eq_glove-loss`

对于权重函数，建议的选择是：
如果$x < c$（例如$c = 100$），则$h(x) = (x/c) ^\alpha$（例如$\alpha = 0.75$）；否则$h(x) = 1$。
在这种情况下，
因为$h(0)=0$，
任何$x_{ij}=0$的平方损失项都可以省略
以提高计算效率。
例如，
当使用小批量随机梯度下降进行训练时，
在每次迭代中
我们随机采样一小批*非零*$x_{ij}$
来计算梯度
并更新模型参数。
注意这些非零$x_{ij}$是预先计算的
全局语料库统计；
因此，该模型被称为GloVe
代表*全局向量*。

应该强调的是
如果词$w_i$出现在词$w_j$的上下文窗口中，则反之亦然。
因此，$x_{ij}=x_{ji}$。
不像word2vec
拟合不对称的条件概率
$p_{ij}$，
GloVe拟合对称的$\log \, x_{ij}$。
因此，在GloVe模型中，任何词的中心词向量和上下文词向量在数学上是等价的。
然而在实践中，由于不同的初始化值，
同一个词在训练后在这两个向量中仍可能得到不同的值：
GloVe将它们相加作为输出向量。



## 从共现概率比的角度解读GloVe


我们也可以从另一个角度解读GloVe模型。
使用:numref:`subsec_skipgram-global`中的相同符号，
令$p_{ij} \stackrel{\textrm{def}}{=} P(w_j \mid w_i)$是在语料库中给定中心词$w_i$生成上下文词$w_j$的条件概率。
:numref:`tab_glove`
列出了根据大型语料库统计得出的
给定词“ice”和“steam”的几个共现概率及其比率。


:Word-word co-occurrence probabilities and their ratios from a large corpus (adapted from Table 1 in :citet:`Pennington.Socher.Manning.2014`)
:label:`tab_glove`

|$w_k$=|solid|gas|water|fashion|
|:--|:-|:-|:-|:-|
|$p_1=P(w_k\mid \textrm{ice})$|0.00019|0.000066|0.003|0.000017|
|$p_2=P(w_k\mid\textrm{steam})$|0.000022|0.00078|0.0022|0.000018|
|$p_1/p_2$|8.9|0.085|1.36|0.96|



我们可以从:numref:`tab_glove`中观察到以下几点：

* 对于与“ice”相关但与“steam”无关的词$w_k$，例如$w_k=\textrm{solid}$，我们期望较大的共现概率比，例如8.9。
* 对于与“steam”相关但与“ice”无关的词$w_k$，例如$w_k=\textrm{gas}$，我们期望较小的共现概率比，例如0.085。
* 对于与“ice”和“steam”都相关的词$w_k$，例如$w_k=\textrm{water}$，我们期望接近1的共现概率比，例如1.36。
* 对于与“ice”和“steam”都不相关的词$w_k$，例如$w_k=\textrm{fashion}$，我们期望接近1的共现概率比，例如0.96。


可以看出，共现概率比
可以直观地表达
词之间的关系。
因此，我们可以设计一个
三个词向量的函数
来拟合这个比率。
对于共现概率比
${p_{ij}}/{p_{ik}}$
其中$w_i$是中心词
而$w_j$和$w_k$是上下文词，
我们希望用某个函数$f$来拟合这个比率：

$$f(\mathbf{u}_j, \mathbf{u}_k, {\mathbf{v}}_i) \approx \frac{p_{ij}}{p_{ik}}.$$
:eqlabel:`eq_glove-f`

在$f$的许多可能设计中，
我们只选择下面的一个合理选择。
由于共现概率比
是一个标量，
我们要求$f$是一个标量函数，例如
$f(\mathbf{u}_j, \mathbf{u}_k, {\mathbf{v}}_i) = f\left((\mathbf{u}_j - \mathbf{u}_k)^\top {\mathbf{v}}_i\right)$。
在:eqref:`eq_glove-f`中交换词索引
$j$和$k$，
必须满足$f(x)f(-x)=1$，
所以一种可能性是$f(x)=\exp(x)$，
即

$$f(\mathbf{u}_j, \mathbf{u}_k, {\mathbf{v}}_i) = \frac{\exp\left(\mathbf{u}_j^\top {\mathbf{v}}_i\right)}{\exp\left(\mathbf{u}_k^\top {\mathbf{v}}_i\right)} \approx \frac{p_{ij}}{p_{ik}}.$$

现在让我们选择
$\exp\left(\mathbf{u}_j^\top {\mathbf{v}}_i\right) \approx \alpha p_{ij}$，
其中$\alpha$是一个常数。
由于$p_{ij}=x_{ij}/x_i$，两边取对数后我们得到$\mathbf{u}_j^\top {\mathbf{v}}_i \approx \log\,\alpha + \log\,x_{ij} - \log\,x_i$。
我们可以使用额外的偏置项来拟合$- \log\, \alpha + \log\, x_i$，例如中心词偏置$b_i$和上下文词偏置$c_j$：

$$\mathbf{u}_j^\top \mathbf{v}_i + b_i + c_j \approx \log\, x_{ij}.$$
:eqlabel:`eq_glove-square`

用权重测量:eqref:`eq_glove-square`的平方误差，
就得到了:eqref:`eq_glove-loss`中的GloVe损失函数。



## 总结

* skip-gram模型可以使用全局语料库统计（如词-词共现计数）来解释。
* 交叉熵损失可能不是衡量两个概率分布差异的好选择，特别是对于大型语料库。GloVe使用平方损失来拟合预计算的全局语料库统计。
* 在GloVe中，任何词的中心词向量和上下文词向量在数学上是等价的。
* GloVe可以从词-词共现概率比的角度来解释。


## 练习

1. 如果词$w_i$和$w_j$在相同的上下文窗口中共现，如何使用它们在文本序列中的距离来重新设计计算条件概率$p_{ij}$的方法？提示：参见GloVe论文:cite:`Pennington.Socher.Manning.2014`的第4.2节。
1. 在GloVe中，任何词的中心词偏置和上下文词偏置在数学上是否等价？为什么？


[讨论](https://discuss.d2l.ai/t/385)